In [12]:
#import sqlite3
import sys
import os
from datetime import datetime, timedelta
import argparse
import importlib
import pandas as pd
import hashlib


import _config as cfg

# 모듈 임포트
import stninfo_to_csv_run as stn
import calc_ct_day_1021 as cal
import _config as cfg

In [5]:

#obs stn 가져오기
strDt = '20240110'
dt = datetime.strptime(strDt, '%Y%m%d')
obs_stn = stn.reqApiAwsInfoData(dt)
print(type(obs_stn))
print(obs_stn)

https://apihub.kma.go.kr/api/typ01/url/stn_inf.php?inf=AWS&stn=&tm=202401100900&help=1&authKey=VsCrv8PCThmAq7_Dwp4ZGA
<class 'list'>
[{'dt': '2024-01-10', 'id': '90', 'lon': 128.56473, 'lat': 38.25085, 'ht': 17.53}, {'dt': '2024-01-10', 'id': '92', 'lon': 128.66298, 'lat': 38.05903, 'ht': 75.4}, {'dt': '2024-01-10', 'id': '93', 'lon': 127.75443, 'lat': 37.94738, 'ht': 95.78}, {'dt': '2024-01-10', 'id': '95', 'lon': 127.3042, 'lat': 38.14787, 'ht': 155.48}, {'dt': '2024-01-10', 'id': '96', 'lon': 131.86983, 'lat': 37.23952, 'ht': 99.08}, {'dt': '2024-01-10', 'id': '98', 'lon': 127.0607, 'lat': 37.90188, 'ht': 115.62}, {'dt': '2024-01-10', 'id': '99', 'lon': 126.76648, 'lat': 37.88589, 'ht': 30.59}, {'dt': '2024-01-10', 'id': '100', 'lon': 128.71834, 'lat': 37.67713, 'ht': 772.43}, {'dt': '2024-01-10', 'id': '101', 'lon': 127.7357, 'lat': 37.90262, 'ht': 75.82}, {'dt': '2024-01-10', 'id': '102', 'lon': 124.71237, 'lat': 37.97396, 'ht': 36.0}, {'dt': '2024-01-10', 'id': '104', 'lon': 128.

In [6]:
obscode = 'aws' # aws,asos,gts
modelcode = 'gdps_ne36' #gdps_ne36, klfs_ne36

xyfile = cfg.PATH_MODEL_STNXY.format(MODEL=modelcode, OBS=obscode)



In [10]:
#비교 -데이터프레임 equals()

if not os.path.exists(xyfile): 
                
    print(f"File does not exist: {xyfile}, file writing.")
    modelStnInfos= stn.model_station_info(dt, obscode, modelcode)
    stn.writeXyCsv(obscode, modelcode, modelStnInfos, dt)
                
                
else: 
    print(f"File exists: {xyfile}, comparing content.")
                
    existingData = pd.read_csv(xyfile, usecols=['stnId', 'lat', 'lon'], dtype={'stnId': int})
    stnApiList_df = pd.DataFrame(obs_stn)
    stnApiList_df = stnApiList_df.rename(columns={'id': 'stnId', 'lat': 'lat', 'lon': 'lon'})
    stnApiList_df = stnApiList_df[['stnId', 'lat', 'lon']]  # 필요한 열만 선택
    stnApiList_df['stnId'] = stnApiList_df['stnId'].astype(int)       
                
    if not existingData.equals(stnApiList_df):
        print(f"Changes detected, updating file: {xyfile}")
        modelStnInfos= stn.model_station_info(dt, obscode, modelcode)
        stn.writeXyCsv(obscode, modelcode, modelStnInfos, dt)
    else:
        print(f"No changes detected, skipping file update: {xyfile}")

File exists: ../DAIN/STNXY/gdps_ne36/stnxy_gdps_ne36_aws.csv, comparing content.
Changes detected, updating file: ../DAIN/STNXY/gdps_ne36/stnxy_gdps_ne36_aws.csv
https://apihub.kma.go.kr/api/typ01/url/stn_inf.php?inf=AWS&stn=&tm=202401100900&help=1&authKey=VsCrv8PCThmAq7_Dwp4ZGA


In [13]:
# 비교 - 해시 기반 비교
def hash_dataframe(df):
    return hashlib.sha256(pd.util.hash_pandas_object(df, index=True).values).hexdigest()

# 두 데이터프레임의 해시값 비교
if hash_dataframe(existingData) != hash_dataframe(stnApiList_df):
    print(f"Changes detected, updating file: {xyfile}")
    modelStnInfos = stn.model_station_info(dt, obscode, modelcode)
    stn.writeXyCsv(obscode, modelcode, modelStnInfos, dt)
else:
    print(f"No changes detected, skipping file update: {xyfile}")



Changes detected, updating file: ../DAIN/STNXY/gdps_ne36/stnxy_gdps_ne36_aws.csv
https://apihub.kma.go.kr/api/typ01/url/stn_inf.php?inf=AWS&stn=&tm=202401100900&help=1&authKey=VsCrv8PCThmAq7_Dwp4ZGA
